# SlidingWindowSampler

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.set_printoptions(precision=4, floatmode="fixed", suppress=True)
rng = np.random.default_rng()

In [ ]:
from torch.utils.data import Sampler

from tsdm.random.samplers import BaseSampler

In [ ]:
from typing import Generic, TypeVar, Union
from collections.abc import Sequence

In [ ]:
from datetime import datetime as py_dt
from datetime import timedelta as py_td
from numpy import datetime64 as np_dt
from numpy import floating as np_float
from numpy import integer as np_int
from numpy import timedelta64 as np_td
from pandas import Timedelta as pd_td
from pandas import Timestamp as pd_dt

TimestampLike = TypeVar("TimestampLike", py_dt, np_dt, pd_dt)
TimedeltaLike = TypeVar("TimedeltaLike", py_td, np_td, pd_td)

TimestampLike = TypeVar("TimestampLike", py_dt, np_dt, pd_dt)
TimedeltaLike = TypeVar("TimedeltaLike", py_td, np_td, pd_td)

TimeLike = TypeVar(
    "TimeLike", int, float, np_int, np_float, py_dt, np_dt, pd_dt, py_td, np_td, pd_td
)

In [ ]:
class SlidingWindowSampler(BaseSampler, Generic[TimeLike]):
    r"""Sampler that generates sliding windows over an interval.

    The `SlidingWindowSampler` generates tuples.

    Inputs:
    - Ordered timestamps T
    - Starting time t_0
    - Final time t_f
    - stride ∆t (how much the sampler advances at each step)
        default, depending on data type of T:
           - integer: GCD(∆T)
           - float: max(⌊AVG(∆T)⌋, ε)
           - timestamp: resolution dependent.
    - horizons: TimeDelta or Tuple[TimeDelta]

    The sampler will return tuples of len(horizons)+1.
    """

    def __init__(
        self,
        data_source: Sequence[TimeLike],
        /,
        *,
        stride: TimeLike,
        horizons: Union[TimeLike, tuple[TimeLike]],
        shuffle: bool = False,
        return_mask: bool = True,
    ):
        super().__init__(data_source)
        self.data = data_source
        self.shuffle = shuffle